In [5]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [6]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [7]:
def policy_eval(policy, env, gamma=1.0, theta=0.00001):
    V = np.zeros(env.nS) # init V with 0s
    
    while True:
        delta = 0 # change in V(s)
        for s in range(env.nS): # calculate E[V(s)]
            v = 0
            for a, a_prob in enumerate(policy[s]):
                for  next_s_prob, next_s, reward, done in env.P[s][a]:
                    v += a_prob * next_s_prob * (reward + gamma*V[next_s])
            delta = max(delta, np.abs(V[s]-v))
            V[s] = v
        if delta < theta:
            break
    return np.array(V)

In [13]:
def policy_improvment(env, policy_eval=policy_eval, gamma=1.0):
    def find_action_values(state, V):
        """Calculate all action values for a state"""
        A = np.zeros(env.nA)
        for a in range(env.nA):
            for prob, next_s, reward, done in env.P[state][a]:
                A[a] += prob * (reward + gamma*V[next_s])
        return A
    
    policy = np.ones([env.nS, env.nA])/env.nA # uniform prob init
    
    while True:
        V = policy_eval(policy, env, gamma)      
        policy_stable = True       
        for s in range(env.nS):
            curr_a = np.argmax(policy[s]) # best under current policy
            new_a = np.argmax(find_action_values(s,V)) # best from V
            
            if curr_a != new_a: # greedy update
                polict_stable = False
            policy[s] = np.eye(env.nA)[new_a] 
        
        if policy_stable:
            return policy, V


In [14]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



Policy Probability Distribution:
[[ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]
 [ 0.25  0.25  0.25  0.25]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]

Value Function:
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

Reshaped Grid Value Function:
[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]



In [15]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)

AssertionError: 
Arrays are not almost equal to 2 decimals

(mismatch 87.5%)
 x: array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.])
 y: array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])